In [1]:
import pandas as pd
import numpy as np
import json, urllib.request
from difflib import get_close_matches

from bokeh.io        import curdoc
from bokeh.layouts   import column, row
from bokeh.models    import (
    GeoJSONDataSource, LinearColorMapper, ColorBar, BasicTicker,
    ColumnDataSource, Select, HoverTool, FactorRange,
    NumeralTickFormatter
)
from bokeh.plotting  import figure
from bokeh.palettes  import Viridis256, Category20c
from bokeh.transform import cumsum
from math            import pi

In [2]:
### ───Step 1) Load & prepare your DataFrame ──────────────────────────────────────
df = pd.read_excel("jobs_in_data_exported.xlsx")
df.columns = df.columns.str.strip()   # remove stray spaces

salary_col = "salary_in_usd"
if salary_col not in df.columns:
    raise KeyError(f"{salary_col!r} not in columns: {df.columns.tolist()}")

In [3]:

df.head()


,work_year,job_title,job_category,salary_currency,salary,salary_in_usd,employee_residence,experience_level,employment_type,work_setting,company_location,company_size
0,2023,Data DevOps Engineer,Data Engineering,EUR,88000,95012,Germany,Mid-level,Full-time,Hybrid,Germany,L
1,2023,Data Architect,Data Architecture and Modeling,USD,186000,186000,United States,Senior,Full-time,In-person,United States,M
2,2023,Data Architect,Data Architecture and Modeling,USD,81800,81800,United States,Senior,Full-time,In-person,United States,M
3,2023,Data Scientist,Data Science and Research,USD,212000,212000,United States,Senior,Full-time,In-person,United States,M
4,2023,Data Scientist,Data Science and Research,USD,93300,93300,United States,Senior,Full-time,In-person,United States,M


In [4]:
### ───Step 2) Chart 1: World‐map choropleth of Salary ────────────────────────────

# fetch GeoJSON
GEOURL    = "https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json"
world_geo = json.loads(urllib.request.urlopen(GEOURL).read().decode())
names      = [feat["properties"]["name"] for feat in world_geo["features"]]

# build a fuzzy map from your df values to geojson names
country_name_map = {}
for c in sorted(df["employee_residence"].unique()):
    if c in names:
        country_name_map[c] = c
    else:
        match = get_close_matches(c, names, n=1, cutoff=0.6)
        country_name_map[c] = match[0] if match else c

# create a MapName column
df["MapName"] = df["employee_residence"].map(country_name_map).fillna(df["employee_residence"])

# init Salary property in geojson
for feat in world_geo["features"]:
    feat["properties"]["Salary"] = 0

geo_src = GeoJSONDataSource(geojson=json.dumps(world_geo))

# dropdowns
sel_country = Select(
    title="Country:", value="All",
    options=["All"] + sorted(df["employee_residence"].unique())
)
sel_spec = Select(
    title="Job Category:", value="All",
    options=["All"] + sorted(df["job_category"].unique())
)

# callback to fill in mean salaries
def update_map(attr, old, new):
    d = df.copy()
    if sel_country.value != "All":
        d = d[d["employee_residence"] == sel_country.value]
    if sel_spec.value != "All":
        d = d[d["job_category"] == sel_spec.value]
    agg = d.groupby("MapName")[salary_col].mean().to_dict()
    for feat in world_geo["features"]:
        nm = feat["properties"]["name"]
        feat["properties"]["Salary"] = float(agg.get(nm, 0))
    geo_src.geojson = json.dumps(world_geo)

sel_country.on_change("value", update_map)
sel_spec .on_change("value", update_map)
update_map(None, None, None)

# color mapper
low, high = df[salary_col].min(), df[salary_col].max()
mapper     = LinearColorMapper(palette=Viridis256, low=low, high=high)

p1 = figure(
    title="Salary by Country",
    tools="pan,wheel_zoom,reset,hover",
    x_axis_location=None, y_axis_location=None,
    width=900, height=500
)
p1.grid.grid_line_color = None
p1.patches(
    "xs", "ys", source=geo_src,
    fill_color={"field":"Salary","transform":mapper},
    line_color="gray", line_width=0.3, fill_alpha=0.8
)
p1.hover.tooltips = [
    ("Country", "@name"),
    ("Salary",  "@Salary{$0,0}")
]
p1.add_layout(
    ColorBar(color_mapper=mapper, ticker=BasicTicker(),
             label_standoff=12, location=(0,0)),
    "right"
)

layout1 = column(
    row(sel_country, sel_spec, sizing_mode="stretch_width"),
    p1
)


In [5]:
### ───Step 3) Chart 2: Salary by Experience Level ────────────────────────────────
bar_df = df.groupby("experience_level")[salary_col].mean().reset_index()
src2   = ColumnDataSource(bar_df)

p2 = figure(
    x_range=bar_df["experience_level"].tolist(),
    title="Salary by Experience Level",
    width=700, height=350,
    tools="hover,reset"
)
p2.vbar(
    x="experience_level", top=salary_col,
    width=0.6, source=src2, fill_color="#2ca02c"
)
p2.yaxis.axis_label = "Salary"
p2.yaxis.formatter  = NumeralTickFormatter(format="$0,0")
p2.hover.tooltips = [
    ("Experience", "@experience_level"),
    ("Salary",     f"@{salary_col}{{${{0,0}}}}")
]

sel2 = Select(
    title="Country (Exp):", value="All",
    options=["All"] + sorted(df["employee_residence"].unique())
)
def update_bar(attr, old, new):
    d = df if sel2.value=="All" else df[df["employee_residence"]==sel2.value]
    u = d.groupby("experience_level")[salary_col].mean().reset_index()
    src2.data = u.to_dict(orient="list")
    p2.x_range.factors = u["experience_level"].tolist()

sel2.on_change("value", update_bar)
layout2 = column(sel2, p2)


In [6]:
### ───Step 4) Chart 3: Salary by Work Setting × Job Category ────────────────────
grp = (
    df
    .groupby(["work_setting","job_category"])[salary_col]
    .mean()
    .reset_index()
)
grp["x"] = list(zip(grp.work_setting, grp.job_category))
src3 = ColumnDataSource(grp)

p3 = figure(
    x_range=FactorRange(*grp["x"]),
    title="Salary by Work Setting × Job Category",
    width=1200, height=400,
    tools="hover,reset"
)
p3.vbar(
    x="x", top=salary_col,
    width=0.6, source=src3, fill_color="#1f77b4"
)
p3.yaxis.axis_label = "Salary"
p3.yaxis.formatter  = NumeralTickFormatter(format="$0,0")
p3.hover.tooltips = [
    ("Category","@x"),
    ("Salary",  f"@{salary_col}{{${{0,0}}}}")
]
p3.xaxis.major_label_orientation   = pi/4
p3.xaxis.major_label_text_font_size = "9pt"
p3.x_range.range_padding           = 0.05

sel3 = Select(
    title="Country (Group):", value="All",
    options=["All"] + sorted(df["employee_residence"].unique())
)
def update_grp(attr, old, new):
    d = df if sel3.value=="All" else df[df["employee_residence"]==sel3.value]
    u = (
        d
        .groupby(["work_setting","job_category"])[salary_col]
        .mean()
        .reset_index()
    )
    u["x"] = list(zip(u.work_setting, u.job_category))
    src3.data = u.to_dict(orient="list")
    p3.x_range.factors = u["x"].tolist()

sel3.on_change("value", update_grp)
layout3 = column(sel3, p3)

In [7]:
### ───Step 5) Chart 4: Salary Trend by Year ──────────────────────────────────────
tdf  = df.groupby("work_year")[salary_col].mean().reset_index()
src4 = ColumnDataSource(tdf)

p4 = figure(
    title="Salary Trend by Year",
    x_axis_label="Year", y_axis_label="Salary",
    width=700, height=350,
    tools="hover,reset"
)
p4.line(
    x="work_year", y=salary_col,
    source=src4, line_width=2, color="#d62728"
)
p4.yaxis.formatter = NumeralTickFormatter(format="$0,0")
p4.hover.tooltips = [
    ("Year",   "@work_year"),
    ("Salary", f"@{salary_col}{{${{0,0}}}}")
]

sel4 = Select(
    title="Job Category (Trend):", value="All",
    options=["All"] + sorted(df["job_category"].unique())
)
def update_trend(attr, old, new):
    d = df if sel4.value=="All" else df[df["job_category"]==sel4.value]
    u = d.groupby("work_year")[salary_col].mean().reset_index()
    src4.data = u.to_dict(orient="list")

sel4.on_change("value", update_trend)
layout4 = column(sel4, p4)



In [8]:
#### ───Step 6) Chart 5: Salary by Employment Type (Pie) ──────────────────────────
emp = df.groupby("employment_type")[salary_col].mean().reset_index()
total = emp[salary_col].sum()
emp["angle"]      = emp[salary_col]/total * 2*pi
emp["percentage"] = emp[salary_col]/total * 100
emp["color"]      = Category20c[len(emp)]
src5 = ColumnDataSource(emp)

p5 = figure(
    title="Salary by Employment Type",
    width=700, height=350,
    tools="reset"
)
p5.wedge(
    x=0, y=1, radius=0.4,
    start_angle=cumsum("angle", include_zero=True),
    end_angle=cumsum("angle"),
    fill_color="color", line_color="white",
    legend_field="employment_type", source=src5
)
p5.axis.visible      = False
p5.grid.grid_line_color = None

sel5 = Select(
    title="Country (Pie):", value="All",
    options=["All"] + sorted(df["employee_residence"].unique())
)
def update_pie(attr, old, new): 
    d = df if sel5.value=="All" else df[df["employee_residence"]==sel5.value]
    e = d.groupby("employment_type")[salary_col].mean().reset_index()
    tot2 = e[salary_col].sum()
    e["angle"]      = e[salary_col]/tot2 * 2*pi
    e["percentage"] = e[salary_col]/tot2 * 100
    e["color"]      = Category20c[len(e)]
    src5.data        = e.to_dict(orient="list")

sel5.on_change("value", update_pie)
layout5 = column(sel5, p5)

In [9]:
### ───Step 7) Chart 6: Salary Distribution Histogram ────────────────────────────
hist, edges = np.histogram(df[salary_col], bins=25)
src6 = ColumnDataSource(dict(top=hist, left=edges[:-1], right=edges[1:]))

p6 = figure(
    title="Salary Distribution",
    x_axis_label="Salary", y_axis_label="Count",
    width=700, height=350,
    tools="hover,reset"
)
p6.quad(
    top="top", bottom=0,
    left="left", right="right",
    source=src6, line_color="white"
)
p6.add_tools(HoverTool(
    tooltips=[("Range","$x{0,0} – $x{0,0}"),("Count","@top")],
    mode="vline"
))
layout6 = column(p6)

In [10]:
### ───Step 8) Add all layouts to the document ────────────────────────────────────
for L in (layout1, layout2, layout3, layout4, layout5, layout6):
    curdoc().add_root(L)

curdoc().title = "Global Salary Dashboard"

In [ ]:
!bokeh serve --show dashboard1.ipynb --port 5010